In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import os
from google.colab import drive 
import re
import zipfile
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/STOR566_FinalProject/data')

Mounted at /content/drive


In [ ]:
def txt_cleaner(string):
    string = re.sub('<[^<]+?>', '', string) #removes html
    string = re.sub(r'[^\w\s]', '', string) #removes everything but letters/numbers and whitespace
    return string.lower() #Makes all characters lowercase
    # Should we implement a method to correct mispellings too? Seems like maybe too much effort...

In [ ]:
#https://github.com/neilpl24/STOR566/data/IMDB Dataset.csv.zip
#https://github.com/neilpl24/STOR566/data/Restaurant_Reviews.tsv

url_test = 'https://raw.githubusercontent.com/neilpl24/STOR566/main/data/Restaurant_Reviews.tsv'
df_test = pd.read_csv(url_test, sep = '\t')
df_test.rename(columns={'Review':'review', 'Liked':'label'}, inplace=True)
df_test["review"] = df_test["review"].apply(txt_cleaner)

In [ ]:
df_test.head()

,review,label
0,wow loved this place,1
1,crust is not good,0
2,not tasty and the texture was just nasty,0
3,stopped by during the late may bank holiday of...,1
4,the selection on the menu was great and so wer...,1


In [ ]:
url_train = "https://raw.githubusercontent.com/neilpl24/STOR566/main/data/IMDB_Dataset.csv"
df_train = pd.read_csv(url_train)

In [ ]:
# Workaround for if Github link isn't working - put .zip file in your working google drive directory

# import zipfile

# zf = zipfile.ZipFile('IMDB Dataset.csv.zip') 
# df_train = pd.read_csv(zf.open('IMDB Dataset.csv'))

In [ ]:
df_train["label"] = 1
for i in range(len(df_train)):
    if df_train.loc[i, "sentiment"] == "negative":
        df_train.loc[i, "label"] = 0
df_train.drop(columns = ["sentiment"], inplace=True)
df_train["review"] = df_train["review"].apply(txt_cleaner)

In [ ]:
df_train.head()

,review,label
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [ ]:
class TextDataset(Dataset):
    """Text dataset."""

    def __init__(self, df, transform=None):
        """
        Args:
            df (DataFrame): dataframe to work with.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label = self.df.iloc[idx, 1]
        review = self.df.iloc[idx, 0]
        #review = np.array([review])
        #sample = {'review': review, 'label': label}

        if self.transform:
            review = self.transform(review)

        return review, label

In [ ]:
# Padding dataFrame vals to length 256 - either cutting to that length or 0-padding up to it
def padding(df, pad_len):
    '''Zero Pad tokens listed under "review" in a dataset to a pad length and return new dataset'''
    new_df = df[["label"]]
    features = []
    for i in range(len(df)):
        feature = np.zeros(pad_len, dtype=int)
        for v in range(min(pad_len, len(df.loc[i, "review"]))):
            feature[v] = df.loc[i, "review"][v]
        features.append(feature)
    new_df['review'] = [features[j] for j in new_df.index]
    return new_df

In [ ]:
def yield_tokens(data_iter):
    for batch, _ in data_iter:
        yield tokenizer(batch)

td = TextDataset(df = df_train)

tokenizer = get_tokenizer('basic_english')
train_it = iter(td)

vocab_later = build_vocab_from_iterator(yield_tokens(train_it), specials=["<unk>"])
vocab_later.set_default_index(vocab_later["<unk>"])

def processing_func(train_df_orig, test_df_orig):
    train_df = train_df_orig
    test_df = test_df_orig

    train_dataset = TextDataset(df = train_df)

    tokenizer = get_tokenizer('basic_english')
    train_iter = iter(train_dataset)

    vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])

    train_df["tokens"] = train_df["review"].apply(lambda x: vocab(tokenizer(x)))
    train_df.drop(["review"], axis = 1, inplace=True)
    train_df.rename(columns = {"tokens":"review"}, inplace=True)
    train_df = train_df[["review", "label"]]

    test_df["tokens"] = test_df["review"].apply(lambda x: vocab(tokenizer(x)))
    test_df.drop(["review"], axis = 1, inplace=True)
    test_df.rename(columns = {"tokens":"review"}, inplace=True)
    test_df = test_df[["review", "label"]]    

    train_df = padding(train_df, 256)
    test_df = padding(test_df, 256)

    return train_df, test_df

AttributeError: ignored

In [ ]:
df_train_tokenized, df_test_tokenized = processing_func(df_train, df_test)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train_iter = TextDataset(df = df_train_tokenized)
train_dataloader = DataLoader(train_iter, batch_size=64, shuffle=False)
test_iter = TextDataset(df_test_tokenized)
test_dataloader = DataLoader(test_iter, batch_size=64, shuffle=False)

cuda


In [ ]:
next(iter(train_dataloader))

[tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
         0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
         1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]),
 tensor([[   27,     4,     1,  ...,    19,   610,     2],
         [    3,   382,   113,  ...,     0,     0,     0],
         [    9,   191,    10,  ...,     0,     0,     0],
         ...,
         [  196,    39,   275,  ...,     0,     0,     0],
         [   37,   619, 36926,  ...,     0,     0,     0],
         [ 1425,   105,   347,  ...,     0,     0,     0]])]

In [ ]:
for zero, batch in enumerate(train_dataloader):
    print(batch)
    break

[tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]), tensor([[   27,     4,     1,  ...,    19,   610,     2],
        [    3,   382,   113,  ...,     0,     0,     0],
        [    9,   191,    10,  ...,     0,     0,     0],
        ...,
        [  196,    39,   275,  ...,     0,     0,     0],
        [   37,   619, 36926,  ...,     0,     0,     0],
        [ 1425,   105,   347,  ...,     0,     0,     0]])]


So as you can kind of see above, the way the batches seem to be coming out is as a list of (0, tensor), where the zero is just a placeholder (not sure why it's there) and tensor that contains 2 subtensors. If you index as "zero, batch in enumerate(dataloader)", during training, you'll get the batch in the tensor. The tensor contents are as follows:

1st sub-tensor: The labels for the batch; 1 is positive, 0 is negative.

2nd sub-tensor: A list of lists, where each list is a length-256 0-padded tokenized review that corresponds to the order of the labels.

This means that each observation can be represented as follows:

Observation i in batch:

reviews=batch[1]

labels=batch[0]

review_i = reviews[i]

label_i = labels[i]

In [ ]:
for data in train_dataloader:
  print(data)
  break

[tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]), tensor([[   27,     4,     1,  ...,    19,   610,     2],
        [    3,   382,   113,  ...,     0,     0,     0],
        [    9,   191,    10,  ...,     0,     0,     0],
        ...,
        [  196,    39,   275,  ...,     0,     0,     0],
        [   37,   619, 36926,  ...,     0,     0,     0],
        [ 1425,   105,   347,  ...,     0,     0,     0]])]


In [ ]:
import torch
import torch.nn as nn
import torchtext.transforms as T
import torchtext.functional as F
from torchtext.datasets import SST2
from torch.utils.data import DataLoader
from torch.hub import load_state_dict_from_url
import torch.optim as optim
import numpy as np
from torchsummary import summary
from torch.autograd import Variable
import matplotlib.pyplot as plt

class SentimentalGRU(nn.Module):
    def __init__(self, vocab_size, output_dim, embedding_dim, hidden_dim, n_layers, 
                 drop_prob=0.5):    
        """
        Initialize the model by setting up the layers
        """
        super().__init__()
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # Embedding and GRU layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, 
                         batch_first=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.4)
        
        # Linear and sigmoid layer
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size()
        
        # Embedding and GRU output
        embedded = self.embedding(x)
        gru_out, hidden = self.gru(embedded, hidden)
        
        # Stack the GRU output
        gru_out = gru_out.contiguous().view(-1, self.hidden_dim)
        
        # Dropout and fully connected layers, with sigmoid transform
        out = self.dropout(gru_out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.fc3(out)
        prob_out = self.sigmoid(out)
        
        prob_out = prob_out.view(batch_size, -1)
        prob_out = prob_out[:, -1]
        
        return prob_out, hidden
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

# Initialize model
gru = GRUModel(256, 128, 3, 1, 0.2)
h = gru.init_hidden()

# Hyperparameters
num_epochs = 2
lr = 0.01

# Set up optimizer and loss function
optimizer = optim.Adam(gru.parameters(), lr=lr)
criterion = nn.BCELoss()

In [ ]:
vocab_sz = vocab_later.__len__() + 1
output_sz = 1
embed_dim = 400
h_dim = 256
n_layer = 2


lstm_mod = SentimentalGRU(vocab_sz, output_sz, embed_dim, h_dim, n_layer)

lr = 0.001
loss_func = nn.BCELoss()
optim = torch.optim.Adam(lstm_mod.parameters(), lr=lr)
num_epochs = 10
clip=5
batch_size = 8

lstm_mod = lstm_mod.to(device)

lstm_mod.train()

epochs = []
epoch_test_losses = []
epoch_train_losses = []

for e in range(num_epochs):
    print("Starting epoch #", e)
    h = lstm_mod.init_hidden(batch_size)
    counter = 0
    for zero, batch in enumerate(train_dataloader):
        labels = batch[0].to(device)
        inputs = batch[1].to(device)
        if (len(labels) != 8):
            print("Encountered shorter batch at batch #", counter)
            break #end of batchs, just leave the last few observations
        h = tuple([each.data for each in h])
        lstm_mod.zero_grad()
        output, h = lstm_mod(inputs, h)
        loss = loss_func(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(lstm_mod.parameters(), clip)
        optim.step()
        counter += 1

    epochs.append(e)
    test_outputs = []
    for zero, batch in enumerate(test_dataloader): 
        labels = batch[0].to(device)
        inputs = batch[1].to(device)
        if (len(labels) != 8):
            break
        h = tuple([each.data for each in h])
        output, h = lstm_mod(inputs, h)
        loss = loss_func(output.squeeze(), labels.float())
        test_outputs.append(float(loss))

    epoch_test_losses.append(np.mean(test_outputs))

    train_outputs = []
    for zero, batch in enumerate(train_dataloader): 
        labels = batch[0].to(device)
        inputs = batch[1].to(device)
        if (len(labels) != 8):
            break
        h = tuple([each.data for each in h])
        output, h = lstm_mod(inputs, h)
        loss = loss_func(output.squeeze(), labels.float())
        train_outputs.append(float(loss))

    epoch_train_losses.append(np.mean(train_outputs))

Epoch 1/2 Done, Total Training Loss: 0.6963470976828309, Total Test Loss: 0.6929317153990269
Epoch 2/2 Done, Total Training Loss: 0.6969500897485582, Total Test Loss: 0.6946677640080452


In [ ]:
total = 0
correct = 0

for data in test_dataloader:
    labels = data[0].unsqueeze(1).float()
    sents = data[1].float()
    preds, _ = gru(sents, h)
    total += labels.size(0)
    correct += (preds.round() == labels).sum()
  
print(f"Test Accuracy = {correct / total}")

Test Accuracy = 0.4959999918937683
